In [1]:
import trainer
from test import env, replay_memory, collector, postprocessor
from preprocess_dqn import process_from_replay_sample as preprocessor
from dqn_agent_test import agent
from transition import Transition

The initialization template is either None or invalid, the servers and jobs will be generated randomly


In [2]:
train_helper = trainer.Trainer(env, agent, collector, replay_memory, postprocessor, preprocessor, Transition, 30)

In [3]:
train_helper.train() 

Episode 0, loss: None, iterations: 0, cum_reward: 0.0


/home/dap/RL/JobScheduling/postprocessor.py:26: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



Episode 1, loss: 5.434026718139648, iterations: 2500, cum_reward: 952478.7533458411
Episode 2, loss: 1.4062137603759766, iterations: 5000, cum_reward: 494935.53300739144
Episode 3, loss: 0.5097237825393677, iterations: 7500, cum_reward: 528371.0371591251
Episode 4, loss: 3.0819404125213623, iterations: 10000, cum_reward: 491771.52833047055
Episode 5, loss: 7.066032409667969, iterations: 12500, cum_reward: 697586.1657519473
Episode 6, loss: 0.7868005037307739, iterations: 15000, cum_reward: 609308.4513438691
Episode 7, loss: 2.095200300216675, iterations: 17500, cum_reward: 524660.2415123706
Episode 8, loss: 1.5753239393234253, iterations: 20000, cum_reward: 312628.0646791961
Episode 9, loss: 1.5989532470703125, iterations: 22500, cum_reward: 404264.58763332217
Episode 10, loss: 2.8631973266601562, iterations: 25000, cum_reward: 622620.1460771986
Episode 11, loss: 0.8044218420982361, iterations: 27500, cum_reward: 506670.2559143393
Episode 12, loss: 3.806842803955078, iterations: 30000,

KeyboardInterrupt: 

In [9]:
action = 1
env.reset()
actions = [0] * 10
for i in range(4998):
    t = Transition(*env.step(action))
    t = postprocessor.process(t)
    t = preprocessor(t)
    action = int(agent.predict(t.next_state_nt))
    actions[action] += 1

In [10]:
actions

[4998, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [12]:
t.state

tensor([[1.0000e+00, 9.4002e+02, 3.8770e+03, 1.1498e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]])

In [18]:
env.get_overview(10000)

{'Total jobs in server queue': 4998,
 'Average response time of each server': [{0: 1938.7086020407223},
  {1: 0.0},
  {2: 0},
  {3: 0},
  {4: 0},
  {5: 0},
  {6: 0},
  {7: 0},
  {8: 0},
  {9: 0}],
 'Average response time accross all servers': 193.87086020407224}

In [30]:
env.get_server_status()[1].status

{'Server id': 1,
 'Type': 'CPU',
 'CPU power': 926.2853998626537,
 'IO power': 834.5001211326271,
 'Job queue length': 1,
 'Expected idle time': 1.1498281082084094,
 'CPU intensive jobs': 0,
 'IO intensive jobs': 1}